# Illustration of predict

In [ ]:
%matplotlib inline

import sys
sys.path.append('../..')

from matplotlib import pylab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

pylab.rcParams['figure.figsize'] = 12, 10

import functools
import numpy
import scipy
import scipy.special

from crocodile.clean import *
from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from arl.test_support import create_named_configuration

Generate baseline coordinates for an observation with a hypothetical north-pole VLA over 6 hours, with a visibility recorded every 10 minutes. The phase center is fixed in the zenith. This results in constant $w$-values of basically zero.

In [ ]:
vlas = create_named_configuration('VLAA_north')
ha_range = numpy.arange(numpy.radians(0),
                        numpy.radians(90),
                        numpy.radians(90 / 36))
dec = numpy.radians(90)
vobs = xyz_to_baselines(vlas.data['xyz'], ha_range, dec)

# Wavelength: 5 metres 
wvl=5
uvw = vobs / wvl

ax = plt.figure().add_subplot(111, projection='3d')
ax.scatter(uvw[:,0], uvw[:,1] , uvw[:,2])
max_uvw = numpy.amax(uvw)
ax.set_xlabel('U [$\lambda$]'); ax.set_xlim((-max_uvw, max_uvw))
ax.set_ylabel('V [$\lambda$]'); ax.set_ylim((-max_uvw, max_uvw))
ax.set_zlabel('W [$\lambda$]'); ax.set_zlim((-max_uvw, max_uvw))
ax.view_init(20, 20);

We then make an image with a grid of sources to generate visibilities from. For reference we also use a direct Fourier Transform to generate visibilities:

In [ ]:
import itertools

theta = 2*0.01
lam = 18000
grid_size = int(theta * lam)
image = numpy.zeros((grid_size, grid_size))

vis = numpy.zeros(len(uvw), dtype=complex)
for l,m in theta/10 * numpy.array(list(itertools.product(range(-3, 4), range(-3, 4)))):
    vis += 1.0*simulate_point(uvw, l, m)
    image[grid_size//2 + int(m*lam),
          grid_size//2 + int(l*lam)] += 1.0
show_image(image, "image", theta)

Now we can attempt to generate visibilities from the image. The quality of this depends quite a bit on the quality of the used anti-aliasing function:

In [ ]:
vis_simple = do_predict(theta, lam, uvw, None, image, simple_predict)
print("Simple: ", numpy.sum(numpy.abs(vis_simple - vis)**2) / numpy.sum(numpy.abs(vis)**2))

oversample = 1024

supports = numpy.arange(2,14)
mrange = numpy.arange(0.9, 2.0, 0.1)
conv_errs = []
for support in supports:
    condition = support
    aaf = anti_aliasing_function(grid_size, 0, support)
    kv1 = kernel_oversample(aaf, oversample, support)
    kv1 /= numpy.sum(kv1[0])
    vis_conv = do_predict(theta, lam, uvw, None, image/numpy.outer(aaf,aaf), conv_predict, kv=kv1)
    conv_errs.append(numpy.sum(numpy.abs(vis_conv - vis)**2))
    print("Convolution %dx%d: " % (support, support),
          numpy.sum(numpy.abs(vis_conv - vis)**2) / numpy.sum(numpy.abs(vis)**2),
          " (mean off-centre", numpy.abs(1-numpy.mean(vis_conv / vis)),")")

# Show how error changes with support
plt.semilogy(supports, conv_errs / numpy.sum(numpy.abs(vis)**2))
plt.xlabel("Support, PSWF c"); plt.ylabel("Error"); plt.show()

# Show error distribution
plt.scatter(uvw[:,0], uvw[:,1], c=numpy.abs(vis_conv - vis))
plt.scatter(-uvw[:,0], -uvw[:,1], c=numpy.abs(vis_conv - vis));